In this file, I try to replicate two cases in BY04 using 1) numerical algorithms; 2) analytical approximation provided in their appendiex. I found

- My results based on numerical methods are very close to the loglinear approximation;
- In the first case w/o stochastic volatility, the results in BY04 are also close, with slightly larger discrepancy from loglinear approximation;
- When there is stochastic volatility, BY04 reported a too high risk premium and too low riskfree rate relative to log linear approximation. They may give the wrong parameter for $\sigma_w$.

# Model Setup and algorithms

The dynamics of the model objects are described by the following equations
$$ \begin{aligned}
x_{t+1}=& \rho x_{t}+\varphi_{e} \sigma_{t} e_{t+1} \\
g_{t+1}=& \mu+x_{t}+\sigma_{t} \eta_{t+1} \\
\sigma_{t+1}^{2}=& \sigma^{2}+v_{1}\left(\sigma_{t}^{2}-\sigma^{2}\right)+\sigma_{w} w_{t+1} \\
\end{aligned}
$$
where $e_{t+1}, \eta_{t+1}, w_{t+1} \sim N.i . i . d .(0,1)$. 

Notice we do not have the separately dividend process as in the original paper.

The law of motions are defined in `src/LRRModel.jl` called `gfunc`, `x'func` and `σ'func`.

Under Epstein-Zin utility function, the SDF is given as: 

$$M_{t+1} = β^θ (\frac{Y_{t+1}}{Y_t})^{-\frac{\theta}{\psi}}R_{t+1}^{\theta-1}$$




## Solve the equilibrium

Therefore, the asset pricing equation is:

$$\begin{aligned}
\mathbb{E}\left[\beta^{\theta}\left(\frac{Y_{t+1}}{Y_{t}}\right)^{-\frac{\theta}{\varphi}}R^{\theta}_{t+1}\right] =& 1\\
\mathbb{E}\left[\beta^{\theta}\exp\left((1-γ)g\right)(\frac{pd(x_{t+1},\sigma_{t+1})+1}{pd(x_{t},\sigma_{t})})^{\theta}\right] =& 1
\end{aligned}
$$
where $pd(x,\sigma)$ is the price-consumption ratio given the states of the economy. 
The second equation follows as $R_{t+1}=\exp(g)\frac{pd(x_{t+1},\sigma_{t+1})+1}{pd(x_{t},\sigma_{t})}$.

To solve this equation, transform it as:
$$pd(x,\sigma) = \mathbb{E}\left[β^{θ}\exp\left((1-γ)g\right)(pd(x',\sigma')+1)^{θ}\right]^{\frac{1}{θ}}$$

which gives a fixed point problem. Therefore, we could start with a guess of $pd(x,\sigma)$ and 
iterate the expectation on the right hand side until converge.

The fixed point problem is defined in `src/LRRModel.jl` called `iteratepd!`.

With P/D ratio, we can solve the riskfree rate $R_{f,t+1}=1/\mathbb{E}[M_{t+1}]$ and 
expected return $\mathbb{E}R_{t+1}$.


# Case I: Time-varying growth rate

The parameterization strictly follow the ones described in paper. Specifically, in the first run we try to replicate the 
result in table II of BY2004 where $\sigma_w=0$.


In [ ]:
cd(@__DIR__)
using Printf, DataFrames
include("../../../src/LRRModel.jl")
using .LRRModel

In [305]:
p1 = LRRParameters(
    β = 0.998, # time discounting at monthly level
    γ = 10.0, # RRA
    φ = 1.5, # IES
    μ = 0.0015, # mean growth rate
    ρ = 0.979, # persistence in the growth rate
    σ̄ = 0.0078, # mean volatility
    φe = 0.044, # loading of x on growth risks
    σw = 0.0, # the volatility of volatility
    ν₁ = 1.0, # persistence of volatility
    
    # # the design of the algorithm is requires more than 1 column in σ; 
    # create two columns to fullfil the requirement. not used in anywyere.
    σmax = 0.0078 + 1e-8,
    σmin = 0.0078 - 1e-8,
    nσ = 3, 
    nx = 30
);

Solve the fixed point problem stated above to obtain pd:

In [2]:
p1.pdmat .= 500 # initial guess
res = fixedpoint((out, x)->iteratepd!(out, x, p1), p1.pdmat, show_trace = false, iterations = 500, m = 4)
p1.pdmat .= res.zero
solve𝔼R!(p1) # solve ER and Rf

The model is simulated at the monthly level. I initialize the economy at $x=0$ and feed in 
randomly drawed shocks and iterate the state variables according to their law of motions.
$R_f$, $\mathbb{E}R$ are calculated along the way by imputation on the state space. 
$R_e = \mathbb{E}R - Rf$ is the risk premium. The function `simulatedmodel` is defined in `src/LRRModel.jl`

In [6]:
sim = simulatemodel(p1, 1000000);

Annualized risk premium (not reported in BY2004):

In [48]:
@printf("Simulated risk premium on consumption: %.2f", mean(sim.Re) * 12 * 100)

Simulated risk premium on consumption: 1.58

By adding long run risk, we indeed generate sizable risk premium on consumptions. With an CRRA with similar parameterization, the risk premium will only be 
$\gamma  σ^2 \approx 0.73%$.


Annualized risk free rate:

In [55]:
@printf("Simulated riskfree rate: %.2f", mean(log.(sim.Rf)) * 100 * 12)

Simulated riskfree rate: 2.58

The average risk-free rate with the same parameterization reported in the paper is 1.34%. 

To verify my result, we can directly compute the analytical approximation to the model using the log linear approximation provided in their appendix:

In [232]:
```
Definition of variables can be found in the appendix with the same symbol.
```function loglinearsolution(param, κ₁ = 0.997, κ₁m = 0.9966)
    @unpack_LRRParameters param
    A1 = (1-1/φ)/(1-κ₁ * ρ) # Eq. A5
    A2 = 0.5 * ((θ - θ/φ)^2 + (θ * A1 * κ₁ * φe)^2)/θ/(1-κ₁ * ν₁) # Eq. A7
    B = κ₁ * A1 * φe # defined above Eq. A9
    λmη = -γ # defined below Eq. A10
    λmw = (1-θ) * A2 * κ₁ # defined below Eq. A10
    λme = (1-θ) * B # defined below Eq. A10
    σa2 = (1 + B^2) * σ̄^2 + (A2 * κ₁ * σw)^2 # Eq. A9, approximated using mean of volatility 

    re = -λmη * σ̄^2 + λme * B * σ̄^2 + κ₁ * A2 * λmw * σw^2 - 0.5 * σa2 # risk premium, A11 
    rf = -log(β) + 1/φ * μ + (1-θ)/θ * re - 1/2/θ * ((λmη^2 + λme^2) * σ̄^2 + λmw^2 * σw^2) # risk free rate, A28

    # below I use the approximation formula to calculate the risk premium on the leveraged dividend (the market return)
    A1m = (ψd - 1/φ)/(1-κ₁m * ρ) # Eq. A16
    βme = κ₁m * A1m * φe # below Eq. A12
    Hm = (λmη^2 + (-λme + βme)^2 + φd^2) # below Eq. A18
    A2m = ((1-θ) * A2 * (1-κ₁ * ν₁) + 0.5 * Hm) / (1-κ₁m * ν₁) # Eq A.20
    βmw = κ₁ * A2m # beloe Eq A.13
    varrm2 = (βme^2 + φd^2) * σ̄^2 + βmw^2 * σw^2 # Eq. A13
    rem = βme * λme * σ̄^2 + βmw * λmw * σw^2 - 0.5 * varrm2 # Eq. A14, risk premium for the market return
    return A1, A2, re, rem, rf
end

# κ₁ depends on the average P/D ratio. They report κ₁ = 0.997 in footnote 4.
# Changes of κ₁ in reasonable range has little effect on the approximation.
z̄1 = mean(log.(sim.pd))
κ₁ = exp(z̄1)/(exp(z̄1) + 1) # footnote 4

z̄1m = log(25.02*12) # from Table IV price-dividen ratio
κ₁m = exp(z̄1m)/(exp(z̄1m) + 1)
p1A1, p1A2, p1re, p1rem, p1rf = loglinearsolution(p1, κ₁, κ₁m);

In [233]:
@printf("Simulated rf: %.2f\n", mean(log.(sim.Rf)) * 100 * 12)
@printf("Approximated rf: %.2f\n", p1rf * 12 * 100)
@printf("Reported rf in Table II: 1.34")

Simulated rf: 2.58
Approximated rf: 2.60
Reported rf in Table II: 1.34

In [234]:
@printf("Simulated risk premium Re: %.2f\n", mean(sim.Re) * 12 * 100)
@printf("Approximated risk premium Re: %.2f", p1re * 12 * 100)

Simulated risk premium Re: 1.58
Approximated risk premium Re: 1.51

As shown above, the numerical results are quite close to the log-linear approximation, and is actually much closer than the number reported in Table II. 

Though we do not calculate the risk premium on the leveraged dividend numerically, the analytical solution provides an approximation to it:

In [235]:
@printf("Equity premium on the market return: %.2f\n", p1rem * 12 * 100)
@printf("Equity premium reported in Table II: 4.20")

Equity premium on the market return: 4.13
Equity premium reported in Table II: 4.20

which is decently close.

If so far the discrepancy between my results and BY04 could be explained by numerical inaccuracy, the comparison of stochastic volatility below shows there is more than that.

# Case II: Stochastic volatility

We parameterize the model in the same way, except now we activate the stochastic volatility , i.e., $\nu_1$ and $\sigma_w$. The parameterization follows Table IV in the paper.

In [292]:
p2 = LRRParameters(
    β = 0.998, # time discounting at monthly level
    γ = 10.0, # RRA
    φ = 1.5, # IES
    μ = 0.0015, # mean growth rate
    ρ = 0.979, # persistence in the growth rate
    σ̄ = 0.0078, # mean volatility
    φe = 0.044, # loading of x on growth risks
    # activate stochastic volatility
    σw = 0.23 * 1e-5, # the volatility of volatility
    ν₁ = 0.987, # persistence of volatility
    nσ = 30, 
    nx = 30
);

In [237]:
p2.pdmat .= 500 # initial guess
res = fixedpoint((out, x)->iteratepd!(out, x, p2), p2.pdmat, iterations = 500, m = 4, ftol = 1e-6)
p2.pdmat .= res.zero;

In [238]:
solve𝔼R!(p2)
sim2 = simulatemodel(p2, 1000000);

In [239]:
z̄1 = mean(log.(sim2.pd))
κ₁ = exp(z̄1)/(exp(z̄1) + 1) # footnote 4

z̄1m = log(25.02*12) # from Table IV price-dividen ratio
κ₁m = exp(z̄1m)/(exp(z̄1m) + 1)

p2A1, p2A2, p2re, p2rem, p2rf = loglinearsolution(p2, κ₁, κ₁m);

Below I compare the effects of the stochastic volatility on risk premia and riskfree rates.

### Comparison of risk premia on consumption

In [250]:
dtre = DataFrame(
    Model = ["σw = 0", "σw = 0.23e-5"], 
    Simulation = round.([mean(sim.Re); mean(sim2.Re)] * 12 * 100, digits = 3), 
    Loglinear = round.([p1re, p2re] * 12 * 100, digits = 3)
)

,Model,Simulation,Loglinear
,String,Float64,Float64
1,σw = 0,1.579,1.515
2,σw = 0.23e-5,1.616,1.552


As shown in the table above, according to my numerical results, $\sigma_w = 0.23 * 10^{-5}$ is not able to generate a large boost to risk premium. This is also consistent with the loglinear approximations. My simulation does not have risk premia on the market return, but we can compare it using loglinear approximations:

### Comparison of risk premia on market returns

In [248]:
dtrem = DataFrame(Model = ["σw = 0", "σw = 0.23e-5"], 
        Loglinear = round.([p1rem, p2rem] * 12 * 100, digits = 3), 
        BY = round.([4.20, 6.84], digits = 3)
    )

,Model,Loglinear,BY
,String,Float64,Float64
1,σw = 0,4.127,4.2
2,σw = 0.23e-5,4.321,6.84


The approximation formula suggests an increase of 0.16% in the risk premia, while BY report a 2.5% increase in Table IV.

### Comparison of riskfree rates

In [246]:
dtrf = DataFrame(
    Model = ["σw = 0", "σw = 0.23e-5"], 
    Simulation = round.([mean(log.(sim.Rf)); mean(log.(sim2.Rf))] * 12 * 100, digits = 3), 
    Loglinear = round.([p1rf, p2rf] * 12 * 100, digits = 3),
    BY = [1.34; 0.93]
)

,Model,Simulation,Loglinear,BY
,String,Float64,Float64,Float64
1,σw = 0,2.584,2.6,1.34
2,σw = 0.23e-5,2.576,2.581,0.93


The stochastic volatility does not generate significant precautionary saving, either, while BY reported a reduction around 40 basis points in the riskfree rate.

# Does my result make sense?

We could use the loglinear approximation to understand how stochastic volatility changes risk premia. The risk premium on consumption is given in A11, copied here:

$$E_{t}\left[r_{a, t+1}-r_{f, t}\right]=-\lambda_{m, \eta} \sigma_{t}^{2}+\lambda_{m, e} B \sigma_{t}^{2}+\kappa_{1} A_{2} \lambda_{m, w} \sigma_{w}^{2}-0.5 var_{t}\left(r_{a, t+1}\right)$$

where $\lambda_{m, \eta}=-\gamma$, $\lambda_{m, e} = (1-\theta)B$, and $B=\kappa_{1} \frac{\varphi_{e}}{1-\kappa_{1} \rho}\left(1-\frac{1}{\psi}\right)$. 

Observe that the parameters governing stochastic volatility does not enter the first two terms. Hence, stochastic volatility can boost risk premium only through the third term $\kappa_{1} A_{2} \lambda_{m, w} \sigma_{w}^{2}$. The expressions for $A_2$ and $\lambda_{m,w}$ are given at A7 and A10:

$$A_{2}=\frac{0.5\left[\left(\theta-\frac{\theta}{\psi}\right)^{2}+\left(\theta A_{1} \kappa_{1} \varphi_{e}\right)^{2}\right]}{\theta\left(1-\kappa_{1} v_{1}\right)}$$

and $\lambda_{m, w} \equiv(1-\theta) A_{2} \kappa_{1}$

In [293]:
@unpack θ, φ, φe, ν₁, σw, ρ, σ̄ = p2;

In [303]:
κ₁ = 0.998
A1 = (1-1/φ)/(1-κ₁ * ρ) # Eq. A5
A2 = 0.5 * ((θ - θ/φ)^2 + (θ * A1 * κ₁ * φe)^2)/θ/(1-κ₁ * ν₁) # Eq. A7
λmw = (1-θ) * A2 * κ₁ # defined below Eq. A10

-13040.204149235058

In [304]:
κ₁ * A2 * λmw * σw^2 * 100 * 12

0.038552066970095035

The term related with stochastic volatility is only 0.05% at the current calibration.

I considered whether they mistyped the magnitude of $\sigma_w$. However, if we use $\sigma_w=2.3\times 10^{-5}$, then we have a way too much larger effect on risk premium: 3.8% increase in consumption.

Another possibility is I misinterpreted the parameters from their paper. Gomez has casted the model in continuous time version, and his interpretation is consistent with mine, and we also get the similar results.